## STRATEGY 

Parameters:
	Stretch:	0.5
	LongTermMALen:	200
	ShortTermMALen:	20
		
Data:
	ATRValue:	ATR(3)
	ADXValue:	ADX(5)
	
	ShortTermMA:	MA(Close, ShortTermMALen)
	LongTermMA:	MA(Close, LongTermMALen)

	ClosingRange:	(Close - Low) / (High - Low)
	IsUptrend: 	Close > LongTermMA
	IsVolatile:	ADXValue > 30			  

	Long_limit:	    Low - (ATRValue * Stretch) // limit order price
	Long_trigger:	ClosingRange < 0.3 // close in lower 30% of bar
	Long_setup:	    LongTrigger and IsUptrend and IsLiquid and IsVolatile
	
	StopLossExit:	Close < ShortTermMA



	Exit_rule:	Close > FillPrice or StopLossExit
	Exit_price:	NextOpen

In [1]:
import vectorbt as vbt
import pandas as pd 
import numpy as np
from datetime import datetime
from kucoin_candle_spot import SpotDataFetcher
from datetime import datetime, timezone
import pandas_ta as ta
from numba import njit
import plotly.graph_objs as go


/Users/andre/Documents/Python_local/mean_reversion_strategy/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## data collection

In [2]:

symbols = ["ETH-USDT", "BTC-USDT"]
timeframe = "1day"
start_time = "2023-01-08 10:00:00"
end_time = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")


# Fetch data for each symbol and store in a dictionary
dataframes = {}
for symbol in symbols:
    fetcher = SpotDataFetcher(symbol, timeframe, start_time, end_time)
    df = fetcher.fetch_candles_as_df()
    df['symbol'] = symbol
    dataframes[symbol] = df

# # Print the DataFrame for each symbol
# for symbol, df in dataframes.items():
#     print(f"\nDataFrame for {symbol}:")
#     print(df)
# fetcher = SpotDataFetcher(symbol, timeframe, start_time, end_time)




INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetching candle data...
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetch complete. Chunks: 2, Candles: 753
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetching candle data...
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetch complete. Chunks: 2, Candles: 753


In [13]:
# data frames is dict of datfarems with candle data for each symbol
print(dataframes)

df_eth = dataframes['ETH-USDT']
df_btc = dataframes['BTC-USDT']

{'ETH-USDT':                               open    close     high      low          volume  \
timestamp                                                                       
2023-01-09 00:00:00+00:00  1290.13  1320.44  1344.79  1285.26  43599.34545334   
2023-01-10 00:00:00+00:00  1320.43  1335.83  1347.08  1317.10  31684.49483084   
2023-01-11 00:00:00+00:00  1335.94  1389.41  1397.56  1321.03  33512.38247916   
2023-01-12 00:00:00+00:00  1389.59  1415.85  1437.73  1356.40  69646.54076749   
2023-01-13 00:00:00+00:00  1415.86  1451.02  1464.51  1400.92  43249.64419696   
...                            ...      ...      ...      ...             ...   
2025-01-26 00:00:00+00:00  3318.69  3233.03  3362.10  3229.85  28348.55197177   
2025-01-27 00:00:00+00:00  3232.71  3182.34  3253.70  3021.15  87840.69636415   
2025-01-28 00:00:00+00:00  3182.51  3077.63  3222.76  3039.78  47969.72429215   
2025-01-29 00:00:00+00:00  3077.67  3114.04  3183.25  3055.00  50313.22874885   
2025-01-30 00:0

In [4]:
def custom_trading_strategy(close, high, low, stretch=0.5, 
                             long_term_ma_len=200, short_term_ma_len=20, adx_len=5, atr_len=3, symbol=None):
    # Calculate indicators
    atr = vbt.IndicatorFactory.from_talib('ATR').run(high, low, close, timeperiod=atr_len).real.to_numpy()
    adx = vbt.IndicatorFactory.from_talib('ADX').run(high, low, close, timeperiod=adx_len).real.to_numpy()
    
    # Moving Averages
    long_term_ma = vbt.IndicatorFactory.from_talib('EMA').run(close, timeperiod=long_term_ma_len).real.to_numpy()
    short_term_ma = vbt.IndicatorFactory.from_talib('EMA').run(close, timeperiod=short_term_ma_len).real.to_numpy()

    # Closing Range
    closing_range = (close - low) / (high - low)
    
    # Conditions
    is_uptrend = close > long_term_ma
    is_volatile = adx > 30
    
    # Long Limit
    long_limit = low - (atr * stretch)
    
    # Long Trigger
    long_trigger = closing_range < 0.3
    
    # Long Setup
    long_setup = long_trigger & is_uptrend & is_volatile
    
    # Return limit order price when conditions for long setup are met
    limit_order_price = np.where(long_setup, long_limit, np.nan)
    
    # Ensure the order of outputs matches output_names
    return limit_order_price, long_term_ma,short_term_ma, atr, adx

# Create the indicator factory
custom_strategy_indicator = vbt.IndicatorFactory(
    class_name='CustomTradingStrategy',
    short_name='custom_strategy',
    input_names=['Close', 'High', 'Low'],
    param_names=['stretch', 'long_term_ma_len','short_term_ma_len', 'adx_len', 'atr_len','symbol'],
    output_names=['limit_order_price' ,'long_term_ma','short_term_ma', 'atr', 'adx']  # Add outputs for indicators

).from_apply_func(custom_trading_strategy)




In [16]:
# Entry parameter 
stretch = 0.5
long_term_ma_len = [100,200]
adx_len = [5,6]
atr_len = 3

# exit parameter
short_term_ma_len = 20


# Usage example (commented out)
indicator = custom_strategy_indicator.run(
    df_btc['close'], df_btc['high'], df_btc['low'],
    stretch=stretch,
    long_term_ma_len=long_term_ma_len,
    short_term_ma_len=short_term_ma_len,
    adx_len=adx_len,
    atr_len=atr_len,
    symbol='BTC-USDT',
    param_product=True

)

df_indicator_signals = indicator.limit_order_price
df_indicator_signals



custom_strategy_stretch                0.5                           
custom_strategy_long_term_ma_len       100               200         
custom_strategy_short_term_ma_len       20                20         
custom_strategy_adx_len                  5        6        5        6
custom_strategy_atr_len                  3        3        3        3
custom_strategy_symbol            BTC-USDT BTC-USDT BTC-USDT BTC-USDT
timestamp                                                            
2023-01-09 00:00:00+00:00              NaN      NaN      NaN      NaN
2023-01-10 00:00:00+00:00              NaN      NaN      NaN      NaN
2023-01-11 00:00:00+00:00              NaN      NaN      NaN      NaN
2023-01-12 00:00:00+00:00              NaN      NaN      NaN      NaN
2023-01-13 00:00:00+00:00              NaN      NaN      NaN      NaN
...                                    ...      ...      ...      ...
2025-01-26 00:00:00+00:00              NaN      NaN      NaN      NaN
2025-01-27 00:00:00+00:00              NaN      NaN      NaN      NaN
2025-01-28 00:00:00+00:00              NaN      NaN      NaN      NaN
2025-01-29 00:00:00+00:00              NaN      NaN      NaN      NaN
2025-01-30 00:00:00+00:00              NaN      NaN      NaN      NaN

[753 rows x 4 columns]

In [12]:
df

,open,close,high,low,volume,turnover,symbol
timestamp,,,,,,,
2023-01-09 00:00:00+00:00,17127.5,17181.4,17394.6,17106.3,5577.37314231,96180461.415334055,BTC-USDT
2023-01-10 00:00:00+00:00,17181.4,17436.5,17495.2,17147.3,3933.07007805,68091218.739313634,BTC-USDT
2023-01-11 00:00:00+00:00,17436.5,17943.2,17999.0,17319.3,4559.96523889,79971307.449813698,BTC-USDT
2023-01-12 00:00:00+00:00,17943.2,18845.6,19102.1,17907.6,10886.19514717,200849541.285974835,BTC-USDT
2023-01-13 00:00:00+00:00,18845.7,19924.9,19994.0,18719.1,9375.409916,180257460.526571197,BTC-USDT
...,...,...,...,...,...,...,...
2025-01-26 00:00:00+00:00,104741.7,102617.4,105511.9,102511.8,876.70195799,91572400.202423731,BTC-USDT
2025-01-27 00:00:00+00:00,102606.7,102083.3,103272.8,97778.8,4766.84840827,478187302.811317367,BTC-USDT
2025-01-28 00:00:00+00:00,102071.6,101336.8,103789.3,100277.7,2133.94324915,218299424.833929149,BTC-USDT


In [17]:
def copy_rename_close_series(df_original, df_indicator_signals):

    def repeat_series_horizontally(series, target_shape):
        return np.tile(series.values.reshape(-1, 1), target_shape[1])

    # Assuming df and df_indicator_signals are already defined
    nedded_copies = len(df_indicator_signals.columns)

    # Repeat the 'close' series horizontally to match the shape of df_indicator_signals
    df_dublicate_close = pd.DataFrame(repeat_series_horizontally(df_original['close'], df_indicator_signals.shape), index=df_original.index)

    # Rename columns if shapes match
    if df_indicator_signals.shape == df_dublicate_close.shape:
        df_dublicate_close.columns = df_indicator_signals.columns
        print("Columns have been renamed")
    else:
        print("The DataFrames do not have the same shape.")
    
    
    return df_dublicate_close

    


df_dublicate_close = copy_rename_close_series(df, df_indicator_signals)
# how to access the columns ----> e.g. df_dublicate_close[(0.5, 100, 20, 5, 3,'BTC-USDT')]
df_dublicate_close



Columns have been renamed


custom_strategy_stretch                 0.5                              
custom_strategy_long_term_ma_len        100                 200          
custom_strategy_short_term_ma_len        20                  20          
custom_strategy_adx_len                   5         6         5         6
custom_strategy_atr_len                   3         3         3         3
custom_strategy_symbol             BTC-USDT  BTC-USDT  BTC-USDT  BTC-USDT
timestamp                                                                
2023-01-09 00:00:00+00:00           17181.4   17181.4   17181.4   17181.4
2023-01-10 00:00:00+00:00           17436.5   17436.5   17436.5   17436.5
2023-01-11 00:00:00+00:00           17943.2   17943.2   17943.2   17943.2
2023-01-12 00:00:00+00:00           18845.6   18845.6   18845.6   18845.6
2023-01-13 00:00:00+00:00           19924.9   19924.9   19924.9   19924.9
...                                     ...       ...       ...       ...
2025-01-26 00:00:00+00:00          102617.4  102617.4  102617.4  102617.4
2025-01-27 00:00:00+00:00          102083.3  102083.3  102083.3  102083.3
2025-01-28 00:00:00+00:00          101336.8  101336.8  101336.8  101336.8
2025-01-29 00:00:00+00:00          103736.1  103736.1  103736.1  103736.1
2025-01-30 00:00:00+00:00          105220.3  105220.3  105220.3  105220.3

[753 rows x 4 columns]

In [7]:
import numpy as np
import vectorbt as vbt
from numba import njit
import talib
from vectorbt.portfolio.enums import SizeType, Direction

# Create arrays to store the data we want to plot
@njit
def order_func_nb(c, high, low, open_, entries, ma_short, entry_price):  # Added entry_price parameter
    close_price = c.close[c.i, c.col]
    close_minus_1bar = c.close[c.i-1, c.col]
    
    # if in position 
    if c.position_now > 0:
        if (close_minus_1bar <= ma_short[c.i-1]) or (close_price > entry_price[c.i]):
            value = vbt.portfolio.nb.order_nb(
                size=-np.inf,
                price=open_[c.i],
                size_type=SizeType.Amount,
                direction=Direction.LongOnly,
                fees=0.001,
                slippage=0.002)
            # Store exit data
            return value

    # if not in position search for position to enter
    elif (c.position_now == 0) and (c.i != 0):
        if (entries[c.i-1,c.col] > 0) and (low[c.i] < entries[c.i-1,c.col]):
            entry_price[:] = np.nan  # Reset entry price array
            entry_price[:] = entries[c.i-1,c.col]  # Update entry price array

            order = vbt.portfolio.nb.order_nb(
                size=1,
                price=entry_price[c.i],
                size_type=SizeType.Percent,
                direction=Direction.LongOnly,
                fees=0.001,
                slippage=0.002,
                allow_partial=False,
                raise_reject=True
            )
            return order

    return vbt.portfolio.enums.NoOrder

# candle data
close = df_dublicate_close
open_ = df['open'].to_numpy().flatten()
high = df['high'].to_numpy().flatten()
low = df['low'].to_numpy().flatten()

# indicator data
entries = df_indicator_signals.to_numpy()
ma_short = indicator.short_term_ma.to_numpy().flatten()

# Create arrays to store data
entry_price = np.full(close.shape[0], np.nan)  # Initialize entry price array

# Create portfolio with trade_data
pf = vbt.Portfolio.from_order_func(
    close,
    order_func_nb,
    high,
    low,
    open_,
    entries,
    ma_short,
    entry_price,  # Pass entry_price array to the function
    init_cash=500
)

In [8]:
total_return = pf.total_return()
max_dd = pf.max_drawdown()
total_return
return_and_maxdd = pd.concat([total_return, max_dd], axis=1)
return_and_maxdd.sort_values(by='total_return', ascending=False).head(50)

total_return  \
custom_strategy_stretch custom_strategy_long_term_ma_len custom_strategy_short_term_ma_len custom_strategy_adx_len custom_strategy_atr_len custom_strategy_symbol                 
0.5                     200                              20                                5                       3                       BTC-USDT                   -0.206312   
                                                                                           6                       3                       BTC-USDT                   -0.216011   
                        100                              20                                5                       3                       BTC-USDT                   -0.218686   
                                                                                           6                       3                       BTC-USDT                   -0.228233   

                                                                                                                                                                   max_drawdown  
custom_strategy_stretch custom_strategy_long_term_ma_len custom_strategy_short_term_ma_len custom_strategy_adx_len custom_strategy_atr_len custom_strategy_symbol                
0.5                     200                              20                                5                       3                       BTC-USDT                   -0.263144  
                                                                                           6                       3                       BTC-USDT                   -0.272148  
                        100                              20                                5                       3                       BTC-USDT                   -0.274631  
                                                                                           6                       3                       BTC-USDT                   -0.283495

In [9]:
trades = pf.trades.records_readable
trades
# combined_df = df.join(trades.set_index('Entry Timestamp'), how='outer')  # Join on index
# combined_df


,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,"(0.5, 100, 20, 5, 3, BTC-USDT)",0.018197,2023-04-21 00:00:00+00:00,27450.076199,0.499500,2023-04-22 00:00:00+00:00,27206.0788,0.495061,-5.434506,-0.010880,Long,Closed,0
1,1,"(0.5, 100, 20, 5, 3, BTC-USDT)",0.017238,2023-05-01 00:00:00+00:00,28662.326916,0.494071,2023-05-02 00:00:00+00:00,28010.2672,0.482831,-12.216886,-0.024727,Long,Closed,1
2,2,"(0.5, 100, 20, 5, 3, BTC-USDT)",0.018365,2023-05-12 00:00:00+00:00,26237.650533,0.481867,2023-05-13 00:00:00+00:00,26743.2064,0.491152,8.311752,0.017249,Long,Closed,2
3,3,"(0.5, 100, 20, 5, 3, BTC-USDT)",0.017999,2023-05-31 00:00:00+00:00,27233.056117,0.490170,2023-06-02 00:00:00+00:00,26760.3720,0.481662,-9.479716,-0.019340,Long,Closed,3
4,4,"(0.5, 100, 20, 5, 3, BTC-USDT)",0.015866,2023-07-05 00:00:00+00:00,30297.088121,0.480700,2023-07-07 00:00:00+00:00,29835.2100,0.473372,-8.282327,-0.017230,Long,Closed,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,63,"(0.5, 200, 20, 6, 3, BTC-USDT)",0.006855,2024-07-25 00:00:00+00:00,64131.357422,0.439624,2024-07-26 00:00:00+00:00,65664.5078,0.450134,9.620071,0.021883,Long,Closed,63
64,64,"(0.5, 200, 20, 6, 3, BTC-USDT)",0.007076,2024-08-01 00:00:00+00:00,63488.813613,0.449234,2024-08-23 00:00:00+00:00,60260.8368,0.426394,-23.716158,-0.052792,Long,Closed,64
65,65,"(0.5, 200, 20, 6, 3, BTC-USDT)",0.006863,2024-08-27 00:00:00+00:00,62002.334014,0.425542,2024-09-19 00:00:00+00:00,61631.3902,0.422996,-3.394444,-0.007977,Long,Closed,65
66,66,"(0.5, 200, 20, 6, 3, BTC-USDT)",0.006506,2024-09-30 00:00:00+00:00,64888.459332,0.422151,2024-10-14 00:00:00+00:00,62746.6552,0.408217,-14.764501,-0.034974,Long,Closed,66


In [10]:
check_ma_200 = ta.ema(df['close'], length=200)
fig = go.Figure()

fig.add_trace(go.Candlestick(
    x=df.index,
    open=df['open'],
    high=df['high'],
    low=df['low'],
    close=df['close'],
    name='candlesticks'))

fig.add_trace(go.Scatter(x=df.index, y=indicator.long_term_ma, mode='lines',marker=dict(color='yellow',size=20), name='200 MA'))

fig.add_trace(go.Scatter(x=df.index, y=indicator.short_term_ma, mode='lines',marker=dict(color='red',size=20), name='20 MA'))

# Add annotations for trades
for i, row in trades.iterrows():
    fig.add_annotation(
        x=row['Entry Timestamp'],
        y=row['Avg Entry Price'],
        text="Entry",
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=-40
    )

for i, row in trades.iterrows():
    fig.add_annotation(
        x=row['Exit Timestamp'],
        y=row['Avg Exit Price'],
        text="Exit",
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=40
    )
# fig.add_trad(go.Scatter(x=df.index, y=entry_exit_bars['entry_time'],))
# Enable y-axis zooming
fig.update_layout(
    yaxis=dict(fixedrange=False),  # Allow y-axis zooming
    clickmode='event+select'  # Enable click events and selection events

)


In [11]:
pf.plot()

/var/folders/33/gyg8_bbj4qn5_tvszhbzjxdm0000gn/T/ipykernel_94846/4031687587.py:1: UserWarning:

Subplot 'orders' raised an exception



TypeError: Only one column is allowed. Use indexing or column argument.